In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('trees_in_real_life').getOrCreate()

22/05/25 14:08:24 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/25 14:08:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/25 14:08:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/25 14:08:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/25 14:08:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/25 14:08:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/25 14:08:32 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/05/2

In [2]:
data = spark.read.csv('College.csv', header=True, inferSchema=True)

In [3]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
for col, dat in zip(data.columns, data.head(1)[0]):
    print(f'{col}: {dat}')

School: Abilene Christian University
Private: Yes
Apps: 1660
Accept: 1232
Enroll: 721
Top10perc: 23
Top25perc: 52
F_Undergrad: 2885
P_Undergrad: 537
Outstate: 7440
Room_Board: 3300
Books: 450
Personal: 2200
PhD: 70
Terminal: 78
S_F_Ratio: 18.1
perc_alumni: 12
Expend: 7041
Grad_Rate: 60


In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [9]:
assembler = VectorAssembler(inputCols=['Apps',
                                     'Accept',
                                     'Enroll',
                                     'Top10perc',
                                     'Top25perc',
                                     'F_Undergrad',
                                     'P_Undergrad',
                                     'Outstate',
                                     'Room_Board',
                                     'Books',
                                     'Personal',
                                     'PhD',
                                     'Terminal',
                                     'S_F_Ratio',
                                     'perc_alumni',
                                     'Expend',
                                     'Grad_Rate'], outputCol='features')

In [10]:
output = assembler.transform(data)

In [14]:
output

DataFrame[School: string, Private: string, Apps: int, Accept: int, Enroll: int, Top10perc: int, Top25perc: int, F_Undergrad: int, P_Undergrad: int, Outstate: int, Room_Board: int, Books: int, Personal: int, PhD: int, Terminal: int, S_F_Ratio: double, perc_alumni: int, Expend: int, Grad_Rate: int, features: vector]

In [11]:
from pyspark.ml.feature import StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [13]:
output_fixed = indexer.fit(output).transform(output)

In [15]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [16]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [17]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [18]:
from pyspark.ml.classification import DecisionTreeClassifier, GBTClassifier, RandomForestClassifier

In [19]:
from pyspark.ml import Pipeline

In [20]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [32]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [33]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [34]:
print('DTC')
print(binary_eval.evaluate(dtc_preds))

DTC
0.9183035714285714


In [35]:
print('RFC')
print(binary_eval.evaluate(rfc_preds))

RFC
0.9872564935064938


In [36]:
print('GBT')
print(binary_eval.evaluate(gbt_preds))

GBT
0.9798701298701299


In [29]:
binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex',
                                            rawPredictionCol='prediction')

In [37]:
print('GBT')
print(binary_eval2.evaluate(gbt_preds))

GBT
0.9429383116883115


In [28]:
gbt_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [38]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [41]:
rfc_acc = acc_eval.evaluate(rfc_preds)

In [42]:
rfc_acc

0.9512195121951219